# Задание 1 (5 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [1]:
!mkdir data
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip -P data
!unzip -o data/corpus_wsd_50k.txt.zip -d data/

--2024-10-01 12:42:15--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_wsd_50k.txt.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip [following]
--2024-10-01 12:42:16--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_wsd_50k.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4723095 (4.5M) [application/zip]
Saving to: ‘data/corpus_wsd_50k.txt.zip’

corpus_wsd_50k.txt. 100%[===================>]   4.50M  --.-KB/s    in 0.09s   

2024-10-01 12:42:16 (

In [2]:
corpus_wsd = []
corpus = open('data/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])
corpus_wsd[1]

[['', 'have', 'Have'],
 ['', 'you', 'you'],
 ['permit%2:41:00::', 'permit', 'permitted'],
 ['', 'it', 'it'],
 ['', 'to', 'to'],
 ['become%2:42:01::', 'become', 'become'],
 ['', 'a', 'a'],
 ['giveaway%1:21:00::', 'giveaway', 'giveaway'],
 ['program%1:09:01::', 'program', 'program'],
 ['rather%4:02:02::', 'rather', 'rather'],
 ['', 'than', 'than'],
 ['', 'one', 'one'],
 ['', 'that', 'that'],
 ['have%2:42:00::', 'have', 'has'],
 ['', 'the', 'the'],
 ['goal%1:09:00::', 'goal', 'goal'],
 ['', 'of', 'of'],
 ['improved%3:00:00::', 'improved', 'improved'],
 ['employee%1:18:00::', 'employee', 'employee'],
 ['morale%1:26:00::', 'morale', 'morale'],
 ['', 'and', 'and'],
 ['', ',', ','],
 ['consequently%4:02:00::', 'consequently', 'consequently'],
 ['', ',', ','],
 ['increased%3:00:00::', 'increased', 'increased'],
 ['productivity%1:07:00::', 'productivity', 'productivity'],
 ['', '?', '?']]

In [3]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
nltk.download('punkt_tab')
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from typing import Union

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
def sentence2bow(sent:Union[str, list]) -> list:
    res = []
    if isinstance(sent, str):
        sent = word_tokenize(sent.lower())
    for token in sent:
        res.append(token.lower())
    return res

In [6]:
def lesk_jaccard(word, context:list):
    max_jaccard_score = 0
    best_jaccard_key = None

    for synset in wn.synsets(word):
        word_key = synset.lemmas()[0].key()
        word_definition = sentence2bow(synset.definition())

        intersection = (set(word_definition) & set(context))
        union = (set(word_definition) | set(context))
        jaccard_score =  len(intersection) / len(union)

        if jaccard_score > max_jaccard_score:
            max_jaccard_score = jaccard_score
            best_jaccard_key = word_key

        return best_jaccard_key

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
all, correct = 0, 0

for doc in corpus_wsd:
  if doc != [['']]:
    sent = sentence2bow([_[1] for _ in doc])
    for word in doc:
      if word[0] != '':
        correct += (lesk_jaccard(word[1].lower(), sent) == word[0]) * 1
        all += 1

accuracy = correct / all

In [9]:
print(f"All: {all}, Correct: {correct}, Accuracy: {accuracy}")

All: 239913, Correct: 61853, Accuracy: 0.25781429101382586


Таким образом, если мы будем приводить текст к нижнему регистру, использовать стоп-слова, токенизировать с помощью word_tokenize() можно получить точность 25.7%

In [10]:
!pip install sentence_transformers transformers accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [11]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
import numpy

sent1 = 'a system of projects or services intended to meet a public need'
sent2 = '(computer science) a sequence of instructions that a computer can interpret and execute'
context = 'i wrote a computer _ that predicts the sense of a word'

sent1_emb = model.encode(sent1)
sent2_emb = model.encode(sent2)
context_emb = model.encode(context)

cosine_sim = cosine_similarity(context_emb.reshape(1, -1), [sent1_emb, sent2_emb])
cosine_sim

array([[0.10057062, 0.3182025 ]], dtype=float32)

Видим, что кос. близость для 2 предложения с контекстом больше, значит предложения семантически ближе друг другу

In [13]:
# Модифицируем предыдущую функцию для кос. сравнения

def lesk_cosine(word, context:str):
    max_similarity_score = 0
    best_key = None

    context_emb = model.encode(context)
    word_keys = []
    definition_embedings = []

    for synset in wn.synsets(word):
        word_keys.append(synset.lemmas()[0].key())
        definition_embedings.append(model.encode(synset.definition()))

    similarity_score = cosine_similarity(context_emb.reshape(1, -1), definition_embedings)[0]
    pairs = list(zip(word_keys, similarity_score))

    for pair in pairs:
        if pair[1] > max_similarity_score:
            max_similarity_score = pair[1]
            best_key = pair[0]

    return best_key

In [14]:
corpus_wsd[3]

[['', 'do', 'Do'],
 ['', 'you', 'you'],
 ['measure%2:31:01::', 'measure', 'measure'],
 ['', 'its', 'its'],
 ['', 'relation', 'relation'],
 ['', 'to', 'to'],
 ['reduced%3:00:04::', 'reduced', 'reduced'],
 ['absenteeism%1:04:00::', 'absenteeism', 'absenteeism'],
 ['', ',', ','],
 ['turnover%1:24:00::', 'turnover', 'turnover'],
 ['', ',', ','],
 ['accident%1:11:01::', 'accident', 'accidents'],
 ['', ',', ','],
 ['', 'and', 'and'],
 ['grievance%1:10:01::', 'grievance', 'grievances'],
 ['', ',', ','],
 ['', 'and', 'and'],
 ['', 'to', 'to'],
 ['improved%5:00:00:better:00', 'improved', 'improved'],
 ['quality%1:07:00::', 'quality', 'quality'],
 ['', 'and', 'and'],
 ['output%1:04:00::', 'output', 'output'],
 ['', '?', '?']]

Возьмём для примера 3-й пример:

In [15]:
all, correct = 0, 0

for doc in [corpus_wsd[3]]:
    if doc != [['']]:
        sent = ' '.join([x[2] for x in doc])
        print(sent)
        for word in doc:
            if word[0] != '':
                try:
                    correct += (lesk_cosine(word[2].lower(), sent) == word[0]) * 1
                    all += 1
                except Exception as e:
                    print(e)


accuracy = correct / all

Do you measure its relation to reduced absenteeism , turnover , accidents , and grievances , and to improved quality and output ?


In [16]:
print(f"All: {all}, Correct: {correct}, Accuracy: {accuracy}")

All: 9, Correct: 4, Accuracy: 0.4444444444444444


Проверив на других примерах, можно увидеть, что точность стала близка к 40%

# Задание 2 (5 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты.

In [26]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering
import numpy as np
from sklearn.metrics import adjusted_rand_score

df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')

In [18]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

for key, _ in grouped_df:
    print(grouped_df.get_group(key), "\n\n")
    break

    word                                            context  gold_sense_id
383  бор  бор ( элемент ) бор — элемент тринадцатой груп...              1
384  бор  бор - углерод - кремний семейство сплавов на о...              1
385  бор  с большим выделением теплоты , образуется окси...              1
386  бор  это объясняется прежде всего тем , что у компл...              1
387  бор  совсем малочисленна . элементарный бор в приро...              1
388  бор  действующем при месторождении горно - химическ...              1
389  бор  b c ) . при нагревании в атмосфере кислорода и...              1
390  бор  собственных минералов бора в чужих минералах о...              1
391  бор  бор - углерод - кремний семейство сплавов на о...              1
392  бор  с другими галогенами с образованием тригалоген...              1
393  бор  и в стабильны и входят в состав природного бор...              1
394  бор  номером . обозначается символом b ( ) . в своб...              1
395  бор  и взаимные пере

In [19]:
def calc_ARI(grouped_df, cluster):
    ARI = []

    for key, _ in grouped_df:
        texts = grouped_df.get_group(key)['context'].values
        X = np.zeros((len(texts), 768))
        for i, text in enumerate(texts):
            X[i] = model.encode(text)

        cluster.fit(X)
        labels = np.array(cluster.labels_) + 1
        ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

    return np.mean(ARI)

In [23]:
kmeans_params = [
    {"n_clusters": 8, "max_iter": 300, "algorithm": "lloyd"},
    {"n_clusters": 8, "max_iter": 300, "algorithm": "elkan"},
    {"n_clusters": 3, "max_iter": 200, "algorithm": "lloyd"},
    {"n_clusters": 3, "max_iter": 500, "algorithm": "lloyd"},
    {"n_clusters": 6, "max_iter": 400, "algorithm": "elkan"}
]

for params in kmeans_params:
    print(params)
    kmeans = KMeans(**params)
    print(calc_ARI(grouped_df, kmeans))

{'n_clusters': 8, 'max_iter': 300, 'algorithm': 'lloyd'}
0.04997843300610455
{'n_clusters': 8, 'max_iter': 300, 'algorithm': 'elkan'}
0.07269807604720076
{'n_clusters': 3, 'max_iter': 200, 'algorithm': 'lloyd'}
0.03990548198688035
{'n_clusters': 3, 'max_iter': 500, 'algorithm': 'lloyd'}
0.0698255967024463
{'n_clusters': 6, 'max_iter': 400, 'algorithm': 'elkan'}
0.07678938548077519


In [24]:
affinity_params = [
    {"damping": 0.5, "max_iter": 200, "affinity": "euclidean", "convergence_iter": 20},
    {"damping": 0.8, "max_iter": 300, "affinity": "euclidean", "convergence_iter": 25},
    {"damping": 0.5, "max_iter": 350, "affinity": "euclidean", "convergence_iter": 35},
    {"damping": 0.7, "max_iter": 200, "affinity": "euclidean", "convergence_iter": 40},
    {"damping": 0.6, "max_iter": 300, "affinity": "euclidean", "convergence_iter": 30}
]

for params in affinity_params:
    print(params)
    affinity = AffinityPropagation(**params)
    print(calc_ARI(grouped_df, affinity))

{'damping': 0.5, 'max_iter': 200, 'affinity': 'euclidean', 'convergence_iter': 20}
0.042740969848549505
{'damping': 0.8, 'max_iter': 300, 'affinity': 'euclidean', 'convergence_iter': 25}
0.04154515818974152
{'damping': 0.5, 'max_iter': 350, 'affinity': 'euclidean', 'convergence_iter': 35}
0.042740969848549505
{'damping': 0.7, 'max_iter': 200, 'affinity': 'euclidean', 'convergence_iter': 40}
0.04154515818974152
{'damping': 0.6, 'max_iter': 300, 'affinity': 'euclidean', 'convergence_iter': 30}
0.042740969848549505


In [25]:
dbscan_params = [
    {"eps": 0.3, "min_samples": 8},
    {"eps": 0.8, "min_samples": 3},
    {"eps": 0.2, "min_samples": 2},
    {"eps": 0.5, "min_samples": 5},
    {"eps": 0.7, "min_samples": 7}
]

for params in dbscan_params:
    print(params, end="\t")
    dbscan = DBSCAN(**params)
    print(calc_ARI(grouped_df, dbscan))

{'eps': 0.3, 'min_samples': 8}	0.0
{'eps': 0.8, 'min_samples': 3}	-0.001784882962278153
{'eps': 0.2, 'min_samples': 2}	0.007404278375247722
{'eps': 0.5, 'min_samples': 5}	-0.011271692824715207
{'eps': 0.7, 'min_samples': 7}	0.05586711174021807


In [28]:
agglomerative_params = [
    {"n_clusters": 3, "linkage": "ward"},
    {"n_clusters": 6, "linkage": "complete"},
    {"n_clusters": 3, "linkage": "average"},
    {"n_clusters": 4, "linkage": "ward"},
    {"n_clusters": 5, "linkage": "ward"}
]

for params in agglomerative_params:
    print(params)
    agglomerative = AgglomerativeClustering(**params)
    print(calc_ARI(grouped_df, agglomerative))

{'n_clusters': 3, 'linkage': 'ward'}	0.034738071269679226
{'n_clusters': 6, 'linkage': 'complete'}	0.07884175664003465
{'n_clusters': 3, 'linkage': 'average'}	-0.009158236277409101
{'n_clusters': 4, 'linkage': 'ward'}	0.038617257915792846
{'n_clusters': 5, 'linkage': 'ward'}	0.03379296274962468


В качестве 5-го алгоритма используем Birch

In [29]:
from sklearn.cluster import Birch

birch_params = [
    {"threshold": 0.5, "branching_factor": 50, "n_clusters": 3},
    {"threshold": 0.3, "branching_factor": 20, "n_clusters": 4},
    {"threshold": 0.5, "branching_factor": 10, "n_clusters": 6},
    {"threshold": 0.8, "branching_factor": 20, "n_clusters": 8},
    {"threshold": 0.1, "branching_factor": 40, "n_clusters": 2},
]

for params in birch_params:
    print(params)
    birch = Birch(**params)
    print(calc_ARI(grouped_df, birch))

{'threshold': 0.5, 'branching_factor': 50, 'n_clusters': 3}


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (2) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(


0.018957481003908684
{'threshold': 0.3, 'branching_factor': 20, 'n_clusters': 4}
0.1329376577946513
{'threshold': 0.5, 'branching_factor': 10, 'n_clusters': 6}


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (5) by BIRCH is less than (6). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (2) by BIRCH is less than (6). Decrease the threshold.
  warnings.warn(


0.04773769732740768
{'threshold': 0.8, 'branching_factor': 20, 'n_clusters': 8}


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (8). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (8). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (8). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:725: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (8). Decrease the threshold.
  warnings.warn(


0.0
{'threshold': 0.1, 'branching_factor': 40, 'n_clusters': 2}
0.014443466080977874
